# Simulação

In [7]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
import json


project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.gcpUtils.auth import getCredentials
from src.gcpUtils.bigQuery import pandasToBq, tableToPandas
from src.gcpUtils.google_storage_manager import *

cred = getCredentials("../bd/planejamento-animale-292719-296d49ccdea6.json")

# Caminho para seu arquivo de distâncias
ARQUIVO_DISTANCIAS = '../dados/distancias_todas_combinacoes_BIDIRECIONAL.csv'

K = 3
K_prime = 6
p = 6
q = 0
C = 0.0225
alpha = 2

ARQUIVO_JSON_FILIAIS = '../dados/filiais_inferior_30.json'
DATA_ANALISE = '2025-08-16'  # Formato AAAA-MM-DD
TABELA_BIGQUERY = 'planejamento-animale-292719.checklists_rollout.ANIMALE_checklist'
PATH_CREDENCAIS = '../bd/planejamento-animale-292719-296d49ccdea6.json'


In [8]:

try:
    print(f"Carregando filiais do arquivo: {ARQUIVO_JSON_FILIAIS}...")
    
    with open(ARQUIVO_JSON_FILIAIS, 'r', encoding='utf-8') as f:
        dados_filiais = json.load(f)
        
    df_filiais = pd.DataFrame(dados_filiais)
    
    lista_nomes_filiais = df_filiais['FILIAL'].unique().tolist()
    
    if not lista_nomes_filiais:
        print("Atenção: Nenhuma filial foi encontrada no arquivo JSON.")
    else:
        print(f"Sucesso. {len(lista_nomes_filiais)} filiais únicas encontradas.")

except FileNotFoundError:
    print(f"Erro: Arquivo JSON não encontrado em '{ARQUIVO_JSON_FILIAIS}'")
    lista_nomes_filiais = []
except Exception as e:
    print(f"Erro ao ler o arquivo JSON: {e}")
    lista_nomes_filiais = []


# --- Montar e Executar a Query ---

if lista_nomes_filiais:
    
    # Ex: Transforma ['Filial A', 'Filial B'] em "('Filial A', 'Filial B')"
    filiais_para_query = "','".join(lista_nomes_filiais)
    filiais_para_query = f"('{filiais_para_query}')"

    query = f"""
        SELECT SKU, FILIAL, VELOCIDADE_VENDA, ALVO, PRESENTE, TRANSITO, 
        EST_TOTAL, CONT_RUPTURA, CONT_FALTA, CONT_EXCESSO, VOLUME_EXCESSO, REGULADOR, EST_DISP, LEADTIME
        FROM {TABELA_BIGQUERY}
        WHERE FILIAL IN {filiais_para_query}
        AND DATA = '{DATA_ANALISE}'
    """
    try:
        # --- Buscar os dados ---
        
        print("Executando consulta no BigQuery...")
        df_resultado = tableToPandas(query, 'planejamento-animale-292719', cred)
        
        print("\n--- Resultado da Consulta ---")
        if df_resultado.empty:
            print("A consulta não retornou dados.")
        else:
            print(df_resultado)

    except Exception as e:
        print(f"\nErro ao executar a consulta no BigQuery: {e}")

else:
    print("\nAnálise não executada pois nenhuma filial foi carregada.")

Carregando filiais do arquivo: ../dados/filiais_inferior_30.json...
Sucesso. 62 filiais únicas encontradas.
Executando consulta no BigQuery...

--- Resultado da Consulta ---
                           SKU                 FILIAL  VELOCIDADE_VENDA  ALVO  \
0        27.03.1447-1511-TAM_1  ANIMALE ALPHAVILLE CM               0.0   1.0   
1       07.20.7684-03032-TAM_1     ANIMALE ARACAJU CM               0.0   2.0   
2        11.05.0565-1802-TAM_1     ANIMALE ARACAJU CM               0.0   2.0   
3        11.05.0572-1419-TAM_1     ANIMALE ARACAJU CM               0.0   2.0   
4        27.03.1447-1511-TAM_1     ANIMALE ARACAJU CM               0.0   1.0   
...                        ...                    ...               ...   ...   
124313  52.13.6217-03032-TAM_1              BUZIOS CM               0.0   2.0   
124314   52.13.6307-0005-TAM_2              BUZIOS CM               0.0   1.0   
124315   52.13.6379-0005-TAM_4              BUZIOS CM               0.0   1.0   
124316  52.15.16

# Simução de passagem dos dias

### Novos parâmetros dos SKU's:
- 1) DISPONIVEL: estoque presente na loja
- 2) TRANSITO: sob o processo de entrega
- 3) ESTOQUE_TOTAL = DISPONIVEL + TRANSITO
- 4) CONT_RUPTURA: produto em estado de ruptura
- 5) VELOCIDADE_VENDA: "fração" de produtos a serem vendidos naquele dia. Quando a velocidade_venda de venda soma 1, há a venda de um produto.
- 6) CONT_EXCESSO: produto em estado de excesso
- 7) VOLUME_EXCESSO: quantidade em execesso
- 8) REGULADOR: quantidade de produtos no centro de distribuição 
- 9) DATA: data daquele registro



In [9]:
from datetime import datetime, timedelta
import math

def carregar_matriz_distancia(caminho_csv):
    """
    Carrega o CSV de distâncias, selecionando apenas as colunas necessárias.
    Retorna um DataFrame vazio em caso de erro.
    """
    colunas_necessarias = ['Filial_A', 'Filial_B', 'Distancia_km']
    try:
        df_dist = pd.read_csv(
            caminho_csv,
            usecols=colunas_necessarias
        )
        print(f"Matriz de distâncias carregada com {len(df_dist)} combinações.")
        return df_dist
    except FileNotFoundError:
        print(f"Erro: Arquivo CSV não encontrado em '{caminho_csv}'")
        return pd.DataFrame(columns=colunas_necessarias)
    except ValueError as e:
        # Erro comum se uma das colunas (usecols) não existir no CSV
        print(f"Erro ao carregar CSV (verifique as colunas): {e}")
        return pd.DataFrame(columns=colunas_necessarias)
    except Exception as e:
        print(f"Erro inesperado ao carregar CSV: {e}")
        return pd.DataFrame(columns=colunas_necessarias)

def _identificar_fontes_destinos(df_estoque):
    # removendo o critério de demanda líquida
    # Identificar Fontes (Onde tem excesso > 0)
    df_fontes = df_estoque[
        df_estoque['VOLUME_EXCESSO'].fillna(0) > 0
    ][['SKU', 'FILIAL', 'VOLUME_EXCESSO']].rename(columns={'FILIAL': 'FILIAL_FONTE'})

    # Identificar Destinos (Ruptura)
    criterio_ruptura = (df_estoque['CONT_RUPTURA'] == 1)
    
    df_destinos = df_estoque[criterio_ruptura][[
        'SKU', 'FILIAL','CONT_RUPTURA', 
        'CONT_FALTA', 'VELOCIDADE_VENDA'
    ]].rename(columns={'FILIAL': 'FILIAL_DESTINO'})

    return df_fontes, df_destinos

def _criar_pares_ranking(df_estoque, df_distancias):
    df_fontes, df_destinos = _identificar_fontes_destinos(df_estoque)
    
    if df_fontes.empty or df_destinos.empty:
        return pd.DataFrame()

    # Produto Cartesiano Fontes x Destinos (por SKU)
    df_pares = pd.merge(df_fontes, df_destinos, on='SKU')
    
    # Removendo auto-transferência
    df_pares = df_pares[df_pares['FILIAL_FONTE'] != df_pares['FILIAL_DESTINO']]
    
    # Merge com distâncias
    df_ranking = pd.merge(
        df_pares,
        df_distancias,
        left_on=['FILIAL_FONTE', 'FILIAL_DESTINO'],
        right_on=['Filial_A', 'Filial_B']
    )
    
    return df_ranking

def calcular_ranking_atual(df_estoque, df_distancias):
    """
    Executa o pipeline de cálculo de ranking para o estado atual do estoque.
    """
    df_ranking = _criar_pares_ranking(df_estoque, df_distancias)
    
    if df_ranking.empty:
        return pd.DataFrame()

    df_ranking['FATOR_PREFERENCIA'] = np.where(
        df_ranking['FILIAL_FONTE'] == "CENTRO DE DISTRIBUICAO", alpha, 1
    )
    
    k_series = np.where(
        df_ranking['FILIAL_DESTINO'].astype(str).str.strip() == "CENTRO DE DISTRIBUICAO",
        K_prime, K
    )

    df_ranking['LEADTIME_CALC'] = C * df_ranking['Distancia_km']
    df_ranking['CUSTO_FRETE'] = k_series * df_ranking['Distancia_km']
    fator_logistica = (1 / (df_ranking['LEADTIME_CALC'] + 1)) + (1 / (df_ranking['CUSTO_FRETE'] + 1))

    # Ranking Final
    df_ranking['RANKING_TRANSFERENCIA'] = fator_logistica *  df_ranking['FATOR_PREFERENCIA']
    
    # Ordenar
    df_ranking = df_ranking.sort_values(by='RANKING_TRANSFERENCIA', ascending=False)
    
    return df_ranking

def selecionar_melhores_transferencias(df_ranking_global):
    """
    Garante apenas 1 fonte (a melhor) para cada destino.
    """
    if df_ranking_global.empty:
        return pd.DataFrame()

    # Pega a primeira ocorrência (maior ranking) para cada par SKU/Destino
    # Como estamos simulando 1 SKU por vez, agrupa apenas por Destino
    melhores = df_ranking_global.groupby(['FILIAL_DESTINO']).first().reset_index()
    
    # Filtra apenas quem tem ranking positivo
    melhores = melhores[melhores['RANKING_TRANSFERENCIA'] > 0]
    
    return melhores


In [10]:
class SimuladorEstoque:
    def __init__(self, df_inicial_db, df_distancias, sku_alvo, data_inicio, velocidade_venda_fixa):
        self.sku = sku_alvo
        self.data_atual = pd.to_datetime(data_inicio)
        self.df_distancias = df_distancias
        self.velocidade_fixa = float(velocidade_venda_fixa)
        
        # --- PREPARAÇÃO DO ESTADO INICIAL ---
        df_sku = df_inicial_db[df_inicial_db['SKU'] == sku_alvo].copy()
        
        # 1. Criação do CD baseada no Regulador
        val_regulador = df_sku['REGULADOR'].max() 
        
        if pd.notna(val_regulador) and val_regulador > 0:
            row_cd = {col: 0 for col in df_sku.columns}
            row_cd['FILIAL'] = 'CENTRO DE DISTRIBUICAO'
            row_cd['SKU'] = self.sku
            row_cd['EST_DISP'] = int(val_regulador)
            row_cd['REGULADOR'] = int(val_regulador)
            row_cd['VELOCIDADE_VENDA'] = 0.0 
            row_cd['ALVO'] = 0 
            row_cd['LEADTIME'] = 0
            
            df_cd = pd.DataFrame([row_cd])
            self.estado = pd.concat([df_sku, df_cd], ignore_index=True)
        else:
            self.estado = df_sku
            
        # 2. Velocidade de Venda
        self.estado['VELOCIDADE_VENDA'] = np.where(
            self.estado['FILIAL'] == 'CENTRO DE DISTRIBUICAO', 
            0.0, 
            self.velocidade_fixa
        )
        
        # 3. Normalização
        cols_float = ['LEADTIME', 'ALVO', 'VOLUME_EXCESSO', 'REGULADOR']
        cols_int = ['EST_DISP', 'TRANSITO', 'EST_TOTAL', 'CONT_RUPTURA', 'CONT_EXCESSO']
        
        for col in cols_float:
            if col in self.estado.columns:
                self.estado[col] = self.estado[col].astype(float).fillna(0.0)
        
        for col in cols_int:
            if col in self.estado.columns:
                self.estado[col] = self.estado[col].astype(int).fillna(0)
        
        if 'ALVO' not in self.estado.columns: self.estado['ALVO'] = 0.0

        # Rastreio
        self.estado['MANDOU_PARA'] = "-"
        self.estado['RECEBENDO_DE'] = "-"
        self.estado['STATUS'] = "-"

        self._atualizar_indicadores_internos()

        # Controles
        self.vendas_acumuladas = {filial: 0.0 for filial in self.estado['FILIAL'].unique()}
        self.fila_transito = []
        self.historico_estados = [] 
        self.log_transferencias = [] 

    def _atualizar_coluna_recebendo_de(self):
        """ Atualiza visualmente quem está recebendo o que (Gerúndio) """
        mapa_origens = {}
        for item in self.fila_transito:
            dest = item['filial_destino']
            orig = item['filial_origem']
            if dest not in mapa_origens: mapa_origens[dest] = set()
            mapa_origens[dest].add(orig)
            
        def get_texto(filial):
            if filial in mapa_origens and mapa_origens[filial]:
                return " / ".join(sorted(list(mapa_origens[filial])))
            return "-"

        self.estado['RECEBENDO_DE'] = self.estado['FILIAL'].apply(get_texto)

    def _resetar_colunas_eventos(self):
        """ Limpa eventos pontuais (mandou/enviou), mantendo status contínuo """
        self.estado['MANDOU_PARA'] = "-"
        self.estado['STATUS'] = np.where(self.estado['TRANSITO'] > 0, "Em Trânsito", "-")

    def _atualizar_indicadores_internos(self):
        self.estado['EST_TOTAL'] = self.estado['EST_DISP'] + self.estado['TRANSITO']
        self.estado['CONT_RUPTURA'] = np.where(self.estado['EST_TOTAL'] == 0, 1, 0)
        self.estado['VOLUME_EXCESSO'] = np.maximum(0, self.estado['EST_TOTAL'] - self.estado['ALVO'])
        self.estado['CONT_EXCESSO'] = np.where(self.estado['VOLUME_EXCESSO'] > 0, 1, 0)
        self.estado['DEMANDA_LIQUIDA'] = np.maximum(0, self.estado['ALVO'] - self.estado['EST_TOTAL'])

    def processar_vendas(self):
        """ Venda sobre estoque que amanheceu na loja """
        for idx, row in self.estado.iterrows():
            filial = row['FILIAL']
            velocidade = row['VELOCIDADE_VENDA']
            
            if velocidade == 0: continue 

            estoque_atual = self.estado.at[idx, 'EST_DISP']
            
            if estoque_atual > 0:
                self.vendas_acumuladas[filial] += velocidade
            
            if self.vendas_acumuladas[filial] >= 1.0:
                qtd_venda = int(self.vendas_acumuladas[filial])
                if estoque_atual >= qtd_venda:
                    self.estado.at[idx, 'EST_DISP'] -= qtd_venda
                    self.vendas_acumuladas[filial] -= qtd_venda
                else:
                    vendeu = estoque_atual
                    self.estado.at[idx, 'EST_DISP'] = 0
                    self.vendas_acumuladas[filial] -= vendeu

    def processar_chegadas_transito(self):
        """ Transito -> Disponivel """
        fila_remanescente = []
        for item in self.fila_transito:
            if item['data_chegada'] <= self.data_atual:
                dest = item['filial_destino']
                idx = self.estado['FILIAL'] == dest
                if idx.any():
                    self.estado.loc[idx, 'TRANSITO'] -= item['qtd']
                    self.estado.loc[idx, 'EST_DISP'] += item['qtd']
                    self.estado.loc[idx, 'TRANSITO'] = self.estado.loc[idx, 'TRANSITO'].clip(lower=0)
                    
                    status_atual = self.estado.loc[idx, 'STATUS'].values[0]
                    texto_add = "Recebido"
                    if status_atual == "-" or status_atual == "Em Trânsito":
                         self.estado.loc[idx, 'STATUS'] = texto_add
                    else:
                         self.estado.loc[idx, 'STATUS'] += f" / {texto_add}"
            else:
                fila_remanescente.append(item)
        self.fila_transito = fila_remanescente

    def _calcular_ranking_do_dia(self):
        df_fontes = self.estado[
            (self.estado['VOLUME_EXCESSO'] > 0) & 
            (self.estado['EST_DISP'] > 0)
        ][['SKU', 'FILIAL', 'VOLUME_EXCESSO']].rename(columns={'FILIAL': 'FILIAL_FONTE'})

        df_destinos = self.estado[
            (self.estado['CONT_RUPTURA'] == 1) | 
            (self.estado['DEMANDA_LIQUIDA'] > 0)
        ][['SKU', 'FILIAL', 'CONT_RUPTURA']].rename(columns={'FILIAL': 'FILIAL_DESTINO'})

        if df_fontes.empty or df_destinos.empty: return pd.DataFrame()

        df_pares = pd.merge(df_fontes, df_destinos, on='SKU')
        df_pares = df_pares[df_pares['FILIAL_FONTE'] != df_pares['FILIAL_DESTINO']]
        
        if df_pares.empty: return pd.DataFrame()

        df_ranking = pd.merge(
            df_pares,
            self.df_distancias,
            left_on=['FILIAL_FONTE', 'FILIAL_DESTINO'],
            right_on=['Filial_A', 'Filial_B']
        )

        if df_ranking.empty: return pd.DataFrame()

        df_ranking['FATOR_PREFERENCIA'] = np.where(
            df_ranking['FILIAL_FONTE'] == "CENTRO DE DISTRIBUICAO", alpha, 1
        )
        k_series = np.where(
            df_ranking['FILIAL_DESTINO'].astype(str).str.strip() == "CENTRO DE DISTRIBUICAO",
            K_prime, K
        )

        df_ranking['LEADTIME_CALC'] = C * df_ranking['Distancia_km']
        df_ranking['CUSTO_FRETE'] = k_series * df_ranking['Distancia_km']
        fator_logistica = (1 / (df_ranking['LEADTIME_CALC'] + 1)) + (1 / (df_ranking['CUSTO_FRETE'] + 1))
        fator_demanda = (df_ranking['CONT_RUPTURA'] * p)
        
        df_ranking['RANKING_TRANSFERENCIA'] = fator_logistica * fator_demanda * df_ranking['FATOR_PREFERENCIA']
        
        return df_ranking.sort_values(by='RANKING_TRANSFERENCIA', ascending=False)

    def executar_transferencias(self):
        df_ranking = self._calcular_ranking_do_dia()
        if df_ranking.empty: return

        melhores = df_ranking[df_ranking['RANKING_TRANSFERENCIA'] > 0].groupby('FILIAL_DESTINO').first().reset_index()

        for _, transf in melhores.iterrows():
            fonte = transf['FILIAL_FONTE']
            destino = transf['FILIAL_DESTINO']
            qtd = 1
            
            idx_fonte = self.estado['FILIAL'] == fonte
            idx_dest = self.estado['FILIAL'] == destino
            est_fonte_atual = self.estado.loc[idx_fonte, 'EST_DISP'].values[0]
            
            if est_fonte_atual >= qtd:
                # 1. Movimenta Físico
                self.estado.loc[idx_fonte, 'EST_DISP'] -= qtd
                self.estado.loc[idx_dest, 'TRANSITO'] += qtd
                
                # --- NOVO: SE FONTE É CD, ATUALIZA REGULADOR DE TODOS ---
                if fonte == "CENTRO DE DISTRIBUICAO":
                    self.estado['REGULADOR'] -= qtd
                    # Garante que não negative visualmente (segurança)
                    self.estado['REGULADOR'] = self.estado['REGULADOR'].clip(lower=0)
                
                # 2. Log Visual: Mandou
                self.estado.loc[idx_fonte, 'MANDOU_PARA'] = destino
                
                status_fonte = self.estado.loc[idx_fonte, 'STATUS'].values[0]
                if "Enviou" not in status_fonte:
                    self.estado.loc[idx_fonte, 'STATUS'] = "Enviou" if status_fonte in ["-", "Em Trânsito"] else status_fonte + " / Enviou"

                status_dest = self.estado.loc[idx_dest, 'STATUS'].values[0]
                if "Em Trânsito" not in status_dest:
                     self.estado.loc[idx_dest, 'STATUS'] = "Em Trânsito" if status_dest == "-" else status_dest + " / Em Trânsito"

                # 3. Fila
                leadtime_dest = self.estado.loc[idx_dest, 'LEADTIME'].values[0]
                dias_transito = math.ceil(leadtime_dest) if leadtime_dest > 0 else 1
                data_chegada = self.data_atual + timedelta(days=dias_transito)
                
                self.fila_transito.append({
                    'filial_destino': destino,
                    'filial_origem': fonte,
                    'qtd': qtd,
                    'data_chegada': data_chegada
                })
                
                self.log_transferencias.append({
                    'Data': self.data_atual.strftime('%Y-%m-%d'),
                    'SKU': self.sku,
                    'Origem': fonte,
                    'Destino': destino,
                    'Qtd': qtd,
                    'Leadtime_Dias': dias_transito,
                    'Previsao_Chegada': data_chegada.strftime('%Y-%m-%d')
                })

    def salvar_estado_diario(self):
        self._atualizar_coluna_recebendo_de()
        
        cols_exportacao = [
            'DATA', 'FILIAL', 'SKU', 
            'EST_DISP', 'TRANSITO', 'EST_TOTAL', 
            'ALVO', 'VELOCIDADE_VENDA', 
            'CONT_RUPTURA', 'CONT_EXCESSO', 'VOLUME_EXCESSO', 'REGULADOR', 
            'LEADTIME', 'TEMPO_RESTANTE_TRANSITO',
            'MANDOU_PARA', 'RECEBENDO_DE', 'STATUS'
        ]
        
        snapshot = self.estado.copy()
        snapshot['DATA'] = self.data_atual
        
        mapa_previsoes = {}
        for item in self.fila_transito:
            filial = item['filial_destino']
            dias = (item['data_chegada'] - self.data_atual).days
            if dias > 0:
                txt = f"{dias} dias ({item['qtd']}un)"
                if filial not in mapa_previsoes: mapa_previsoes[filial] = []
                mapa_previsoes[filial].append(txt)
            
        snapshot['TEMPO_RESTANTE_TRANSITO'] = snapshot['FILIAL'].apply(
            lambda x: " / ".join(mapa_previsoes[x]) if x in mapa_previsoes else "-"
        )
        
        cols_finais = [c for c in cols_exportacao if c in snapshot.columns]
        self.historico_estados.append(snapshot[cols_finais])

    def executar_simulacao(self, dias=1):
        print(f"--- Simulação: {self.sku} ({dias} dias) ---")
        self.salvar_estado_diario() 
        for d in range(1, dias + 1):
            self.data_atual += timedelta(days=1)
            self._resetar_colunas_eventos()
            self.processar_vendas()
            self.processar_chegadas_transito()
            self._atualizar_indicadores_internos()
            self.executar_transferencias()
            self.salvar_estado_diario()
        print("--- Simulação Concluída ---")

    def exportar_resultados(self):
        if not self.historico_estados: return pd.DataFrame(), pd.DataFrame()
        return pd.concat(self.historico_estados, ignore_index=True), pd.DataFrame(self.log_transferencias)

In [11]:
class SimuladorEstoque:
    def __init__(self, df_inicial_db, df_distancias, sku_alvo, data_inicio, velocidade_venda_fixa):
        self.sku = sku_alvo
        self.data_atual = pd.to_datetime(data_inicio)
        self.df_distancias = df_distancias
        self.velocidade_fixa = float(velocidade_venda_fixa)
        
        # --- PREPARAÇÃO DO ESTADO INICIAL ---
        df_sku = df_inicial_db[df_inicial_db['SKU'] == sku_alvo].copy()
        
        # 1. Criação do CD baseada no Regulador
        val_regulador = df_sku['REGULADOR'].max() 
        
        if pd.notna(val_regulador) and val_regulador > 0:
            row_cd = {col: 0 for col in df_sku.columns}
            row_cd['FILIAL'] = 'CENTRO DE DISTRIBUICAO'
            row_cd['SKU'] = self.sku
            row_cd['EST_DISP'] = int(val_regulador)
            row_cd['REGULADOR'] = int(val_regulador)
            row_cd['VELOCIDADE_VENDA'] = 0.0 
            row_cd['ALVO'] = 0 
            row_cd['LEADTIME'] = 0
            
            df_cd = pd.DataFrame([row_cd])
            self.estado = pd.concat([df_sku, df_cd], ignore_index=True)
        else:
            self.estado = df_sku
            
        # 2. Velocidade de Venda
        self.estado['VELOCIDADE_VENDA'] = np.where(
            self.estado['FILIAL'] == 'CENTRO DE DISTRIBUICAO', 
            0.0, 
            self.velocidade_fixa
        )
        
        # 3. Normalização
        cols_float = ['LEADTIME', 'ALVO', 'VOLUME_EXCESSO', 'REGULADOR']
        cols_int = ['EST_DISP', 'TRANSITO', 'EST_TOTAL', 'CONT_RUPTURA', 'CONT_EXCESSO']
        
        for col in cols_float:
            if col in self.estado.columns:
                self.estado[col] = self.estado[col].astype(float).fillna(0.0)
        
        for col in cols_int:
            if col in self.estado.columns:
                self.estado[col] = self.estado[col].astype(int).fillna(0)
        
        # O ALVO é mantido internamente APENAS para calcular o Excesso corretamente
        if 'ALVO' not in self.estado.columns: self.estado['ALVO'] = 0.0

        # Rastreio
        self.estado['MANDOU_PARA'] = "-"
        self.estado['RECEBENDO_DE'] = "-"
        self.estado['STATUS'] = "-"

        self._atualizar_indicadores_internos()

        # Controles
        self.vendas_acumuladas = {filial: 0.0 for filial in self.estado['FILIAL'].unique()}
        self.fila_transito = []
        self.historico_estados = [] 
        self.log_transferencias = [] 

    def _atualizar_coluna_recebendo_de(self):
        """ Atualiza visualmente quem está recebendo o que (Gerúndio) """
        mapa_origens = {}
        for item in self.fila_transito:
            dest = item['filial_destino']
            orig = item['filial_origem']
            if dest not in mapa_origens: mapa_origens[dest] = set()
            mapa_origens[dest].add(orig)
            
        def get_texto(filial):
            if filial in mapa_origens and mapa_origens[filial]:
                return " / ".join(sorted(list(mapa_origens[filial])))
            return "-"

        self.estado['RECEBENDO_DE'] = self.estado['FILIAL'].apply(get_texto)

    def _resetar_colunas_eventos(self):
        """ Limpa eventos pontuais (mandou/enviou), mantendo status contínuo """
        self.estado['MANDOU_PARA'] = "-"
        self.estado['STATUS'] = np.where(self.estado['TRANSITO'] > 0, "Em Trânsito", "-")

    def _atualizar_indicadores_internos(self):
        self.estado['EST_TOTAL'] = self.estado['EST_DISP'] + self.estado['TRANSITO']
        
        # Ruptura: Apenas se Total for 0
        self.estado['CONT_RUPTURA'] = np.where(self.estado['EST_TOTAL'] == 0, 1, 0)
        
        # Excesso: Mantido conforme original (Total - Alvo)
        # Necessário para definir FONTES
        self.estado['VOLUME_EXCESSO'] = np.maximum(0, self.estado['EST_TOTAL'] - self.estado['ALVO'])
        self.estado['CONT_EXCESSO'] = np.where(self.estado['VOLUME_EXCESSO'] > 0, 1, 0)
        
        # Demanda Liquida: REMOVIDA
        if 'DEMANDA_LIQUIDA' in self.estado.columns:
            self.estado = self.estado.drop(columns=['DEMANDA_LIQUIDA'])

    def processar_vendas(self):
        """ Venda sobre estoque que amanheceu na loja """
        for idx, row in self.estado.iterrows():
            filial = row['FILIAL']
            velocidade = row['VELOCIDADE_VENDA']
            
            if velocidade == 0: continue 

            estoque_atual = self.estado.at[idx, 'EST_DISP']
            
            if estoque_atual > 0:
                self.vendas_acumuladas[filial] += velocidade
            
            if self.vendas_acumuladas[filial] >= 1.0:
                qtd_venda = int(self.vendas_acumuladas[filial])
                if estoque_atual >= qtd_venda:
                    self.estado.at[idx, 'EST_DISP'] -= qtd_venda
                    self.vendas_acumuladas[filial] -= qtd_venda
                else:
                    vendeu = estoque_atual
                    self.estado.at[idx, 'EST_DISP'] = 0
                    self.vendas_acumuladas[filial] -= vendeu

    def processar_chegadas_transito(self):
        """ Transito -> Disponivel """
        fila_remanescente = []
        for item in self.fila_transito:
            if item['data_chegada'] <= self.data_atual:
                dest = item['filial_destino']
                idx = self.estado['FILIAL'] == dest
                if idx.any():
                    self.estado.loc[idx, 'TRANSITO'] -= item['qtd']
                    self.estado.loc[idx, 'EST_DISP'] += item['qtd']
                    self.estado.loc[idx, 'TRANSITO'] = self.estado.loc[idx, 'TRANSITO'].clip(lower=0)
                    
                    status_atual = self.estado.loc[idx, 'STATUS'].values[0]
                    texto_add = "Recebido"
                    if status_atual == "-" or status_atual == "Em Trânsito":
                         self.estado.loc[idx, 'STATUS'] = texto_add
                    else:
                         self.estado.loc[idx, 'STATUS'] += f" / {texto_add}"
            else:
                fila_remanescente.append(item)
        self.fila_transito = fila_remanescente

    def _calcular_ranking_do_dia(self):
        # 1. Fontes: Tem Excesso e Estoque Disponível > 0
        # (Mantendo a lógica original de Fonte baseada em Excesso)
        df_fontes = self.estado[
            (self.estado['VOLUME_EXCESSO'] > 0) & 
            (self.estado['EST_DISP'] > 0)
        ][['SKU', 'FILIAL', 'VOLUME_EXCESSO']].rename(columns={'FILIAL': 'FILIAL_FONTE'})

        # 2. Destinos: APENAS Ruptura (Estoque Total == 0)
        # (Demanda Líquida foi removida da equação)
        df_destinos = self.estado[
            self.estado['CONT_RUPTURA'] == 1
        ][['SKU', 'FILIAL', 'CONT_RUPTURA']].rename(columns={'FILIAL': 'FILIAL_DESTINO'})

        if df_fontes.empty or df_destinos.empty: return pd.DataFrame()

        df_pares = pd.merge(df_fontes, df_destinos, on='SKU')
        df_pares = df_pares[df_pares['FILIAL_FONTE'] != df_pares['FILIAL_DESTINO']]
        
        if df_pares.empty: return pd.DataFrame()

        df_ranking = pd.merge(
            df_pares,
            self.df_distancias,
            left_on=['FILIAL_FONTE', 'FILIAL_DESTINO'],
            right_on=['Filial_A', 'Filial_B']
        )

        if df_ranking.empty: return pd.DataFrame()

        df_ranking['FATOR_PREFERENCIA'] = np.where(
            df_ranking['FILIAL_FONTE'] == "CENTRO DE DISTRIBUICAO", alpha, 1
        )
        k_series = np.where(
            df_ranking['FILIAL_DESTINO'].astype(str).str.strip() == "CENTRO DE DISTRIBUICAO",
            K_prime, K
        )

        df_ranking['LEADTIME_CALC'] = C * df_ranking['Distancia_km']
        df_ranking['CUSTO_FRETE'] = k_series * df_ranking['Distancia_km']
        fator_logistica = (1 / (df_ranking['LEADTIME_CALC'] + 1)) + (1 / (df_ranking['CUSTO_FRETE'] + 1))
        
        # Fator Demanda considera apenas a Ruptura
        fator_demanda = (df_ranking['CONT_RUPTURA'] * p)
        
        df_ranking['RANKING_TRANSFERENCIA'] = fator_logistica * fator_demanda * df_ranking['FATOR_PREFERENCIA']
        
        return df_ranking.sort_values(by='RANKING_TRANSFERENCIA', ascending=False)

    def executar_transferencias(self):
        df_ranking = self._calcular_ranking_do_dia()
        if df_ranking.empty: return

        melhores = df_ranking[df_ranking['RANKING_TRANSFERENCIA'] > 0].groupby('FILIAL_DESTINO').first().reset_index()

        for _, transf in melhores.iterrows():
            fonte = transf['FILIAL_FONTE']
            destino = transf['FILIAL_DESTINO']
            qtd = 1
            
            idx_fonte = self.estado['FILIAL'] == fonte
            idx_dest = self.estado['FILIAL'] == destino
            est_fonte_atual = self.estado.loc[idx_fonte, 'EST_DISP'].values[0]
            
            if est_fonte_atual >= qtd:
                # 1. Movimenta Físico
                self.estado.loc[idx_fonte, 'EST_DISP'] -= qtd
                self.estado.loc[idx_dest, 'TRANSITO'] += qtd
                
                # Sincroniza Regulador Globalmente se Fonte for CD
                if fonte == "CENTRO DE DISTRIBUICAO":
                    self.estado['REGULADOR'] -= qtd
                    self.estado['REGULADOR'] = self.estado['REGULADOR'].clip(lower=0)
                
                # 2. Log Visual: Mandou
                self.estado.loc[idx_fonte, 'MANDOU_PARA'] = destino
                
                status_fonte = self.estado.loc[idx_fonte, 'STATUS'].values[0]
                if "Enviou" not in status_fonte:
                    self.estado.loc[idx_fonte, 'STATUS'] = "Enviou" if status_fonte in ["-", "Em Trânsito"] else status_fonte + " / Enviou"

                status_dest = self.estado.loc[idx_dest, 'STATUS'].values[0]
                if "Em Trânsito" not in status_dest:
                     self.estado.loc[idx_dest, 'STATUS'] = "Em Trânsito" if status_dest == "-" else status_dest + " / Em Trânsito"

                # 3. Fila
                leadtime_dest = self.estado.loc[idx_dest, 'LEADTIME'].values[0]
                dias_transito = math.ceil(leadtime_dest) if leadtime_dest > 0 else 1
                data_chegada = self.data_atual + timedelta(days=dias_transito)
                
                self.fila_transito.append({
                    'filial_destino': destino,
                    'filial_origem': fonte,
                    'qtd': qtd,
                    'data_chegada': data_chegada
                })
                
                self.log_transferencias.append({
                    'Data': self.data_atual.strftime('%Y-%m-%d'),
                    'SKU': self.sku,
                    'Origem': fonte,
                    'Destino': destino,
                    'Qtd': qtd,
                    'Leadtime_Dias': dias_transito,
                    'Previsao_Chegada': data_chegada.strftime('%Y-%m-%d')
                })

    def salvar_estado_diario(self):
        self._atualizar_coluna_recebendo_de()
        
        # EXPORTAÇÃO: Excesso fica, Alvo e Demanda removidos
        cols_exportacao = [
            'DATA', 'FILIAL', 'SKU', 
            'EST_DISP', 'TRANSITO', 'EST_TOTAL', 
            'VELOCIDADE_VENDA', 'CONT_RUPTURA', 
            'VOLUME_EXCESSO', 'CONT_EXCESSO', # Mantidos
            'REGULADOR', 'LEADTIME', 'TEMPO_RESTANTE_TRANSITO',
            'MANDOU_PARA', 'RECEBENDO_DE', 'STATUS'
        ]
        
        snapshot = self.estado.copy()
        snapshot['DATA'] = self.data_atual
        
        mapa_previsoes = {}
        for item in self.fila_transito:
            filial = item['filial_destino']
            dias = (item['data_chegada'] - self.data_atual).days
            if dias > 0:
                txt = f"{dias} dias ({item['qtd']}un)"
                if filial not in mapa_previsoes: mapa_previsoes[filial] = []
                mapa_previsoes[filial].append(txt)
            
        snapshot['TEMPO_RESTANTE_TRANSITO'] = snapshot['FILIAL'].apply(
            lambda x: " / ".join(mapa_previsoes[x]) if x in mapa_previsoes else "-"
        )
        
        cols_finais = [c for c in cols_exportacao if c in snapshot.columns]
        self.historico_estados.append(snapshot[cols_finais])

    def executar_simulacao(self, dias=1):
        print(f"--- Simulação: {self.sku} ({dias} dias) ---")
        self.salvar_estado_diario() 
        for d in range(1, dias + 1):
            self.data_atual += timedelta(days=1)
            self._resetar_colunas_eventos()
            self.processar_vendas()
            self.processar_chegadas_transito()
            self._atualizar_indicadores_internos()
            self.executar_transferencias()
            self.salvar_estado_diario()
        print("--- Simulação Concluída ---")

    def exportar_resultados(self):
        if not self.historico_estados: return pd.DataFrame(), pd.DataFrame()
        return pd.concat(self.historico_estados, ignore_index=True), pd.DataFrame(self.log_transferencias)

In [12]:
df_distancias = carregar_matriz_distancia(ARQUIVO_DISTANCIAS)
df_estoque_dia = df_resultado.copy()

# 2. Configurar Parâmetros
sku_teste = '27.03.1447-1511-TAM_1' # Exemplo da imagem
dias_simulacao = 5
data_inicio = '2025-09-08'

simulador = SimuladorEstoque(
    df_inicial_db=df_estoque_dia,   # Seu DataFrame vindo do BigQuery
    df_distancias=df_distancias,        # Seu DataFrame de distâncias
    sku_alvo='27.03.1447-1511-TAM_1',     # SKU escolhido
    data_inicio='2025-09-08',     # Data inicial
    velocidade_venda_fixa=0.5     # Input manual de teste
)
simulador.executar_simulacao(dias=30)
df_final, df_log = simulador.exportar_resultados()

# salvar resultados na excel
with pd.ExcelWriter('simulacao_estoque_resultados.xlsx') as writer:
    df_final.to_excel(writer, sheet_name='Estados_Diarios', index=False)
    df_log.to_excel(writer, sheet_name='Log_Transferencias', index=False)



Matriz de distâncias carregada com 3906 combinações.
--- Simulação: 27.03.1447-1511-TAM_1 (30 dias) ---
--- Simulação Concluída ---
